# How it works?
- We install Conda in Colab, then create conda environment with Python3.8 and Poetry
- Then we run new Jupyter in conda + Poetry environment
- Access to new Jupyter is done via ngrok.com SSH forwarding
- In order for SSH forwarding to work smoothly and more or less securely I stored SSH configuration on Google Drive
- You should configurate [SSH Public Keys](https://dashboard.ngrok.com/tunnel-agents/ssh-keys) on ngrok.com
- Have a look at my SSH configuration below. Here
 - ```id_ed25519 / id_ed25519.pub``` - Private and public SSH-key for [Tunnel Agents](https://dashboard.ngrok.com/tunnel-agents/ssh-keys) in ngrok.com
 - ```config``` - SSH authentication for [Tunnel Agents](https://dashboard.ngrok.com/tunnel-agents/ssh-keys) in ngrok.com
 - ```authorized_keys``` - List of Public keys from my local machines

In [1]:
! ls /content/drive/MyDrive/projects/qmlcourse/config/

authorized_keys  config  id_ed25519  id_ed25519.pub


In [2]:
! cat /content/drive/MyDrive/projects/qmlcourse/config/config

Host *.ngrok.com
  IdentityFile /content/drive/MyDrive/projects/qmlcourse/config/id_ed25519
  StrictHostKeyChecking no


# Configuration

In [3]:
SSH_CONFIG_DIR = "/content/drive/MyDrive/projects/qmlcourse/config"

CONDA_ENVIRONMENT = "qmlcourse"
CONDA_ENV_PATH = f"/content/anaconda3/envs/{CONDA_ENVIRONMENT}"

# Install Anaconda

In [4]:
%%bash
wget -q https://repo.anaconda.com/archive/Anaconda3-2021.05-Linux-x86_64.sh
/bin/bash ./Anaconda3-2021.05-Linux-x86_64.sh -b -p /content/anaconda3 >/content/01_install_anaconda.log
rm -rf /content/Anaconda3-2021.05-Linux-x86_64.sh /content/sample_data
/content/anaconda3/bin/conda update -n base -c defaults conda >/content/02_conda_update.log

## Install Python 3.8 + Poetry

In [5]:
%%bash -s "$CONDA_ENVIRONMENT"
/content/anaconda3/bin/conda create -n ${1} python=3.8 poetry --yes >/content/03_conda_create.log

# Clone Projects

In [6]:
%%bash

git clone https://github.com/SemyonSinchenko/qmlcourse
git clone https://github.com/chiant1/qmlcourse_walkthrough

Cloning into 'qmlcourse'...
Cloning into 'qmlcourse_walkthrough'...


# Install Poetry Environment

In [7]:
%%bash -s "$CONDA_ENV_PATH"
cd qmlcourse && PYTHONPATH=${1}/lib/python3.8/site-packages ${1}/bin/poetry install >/content/04_poetry_install.log

# Install, Configure and Start SSH

In [8]:
%%bash -s "$SSH_CONFIG_DIR"
apt-get install -y ssh tmux vim >/content/05_install_ssh.log

# Configurate SSH
mkdir -p /root/.ssh
cp ${1}/config ${HOME}/.ssh  
cp ${1}/authorized_keys ${HOME}/.ssh  

# Restart SSH Service
service ssh restart

 * Restarting OpenBSD Secure Shell server sshd
   ...done.


# Run Jupyter & SSH Port Forwarding (via ngrok)

In [9]:
%%bash -s "$CONDA_ENV_PATH"

# "Fix" google.colab extensions :)
rm -f /etc/ipython/ipython_config.py /etc/jupyter/jupyter_notebook_config.py

# Run Jupyter Notebook
cd qmlcourse && PYTHONPATH=${1}/lib/python3.8/site-packages \
  nohup ${1}/bin/poetry run jupyter notebook \
  --no-browser --notebook-dir=/content --allow-root --ip=localhost --port=8888 \
  --NotebookApp.token='' \
  --NotebookApp.nbserver_extensions="{'google.colab._serverextension':False}" >/content/06_run_jupyter.log 2>&1 &

# Set up ngrok SSH tunnel
ssh -R 0:localhost:22 tunnel.us.ngrok.com tcp 22


Process is interrupted.


# What's next?
- Go to [ngrok.com / Endpoints](https://dashboard.ngrok.com/endpoints/status). Find your Endpoint URL. For example: **```tcp://6.tcp.ngrok.io:12345```**
- Set up SSH Local port Forwarding for Jupyter. For the example above: **```ssh -L 8888:127.0.0.1:8888 -o StrictHostKeyChecking=no -o "UserKnownHostsFile /dev/null" -oPort=12345 6.tcp.ngrok.io```**
- Go to [Jupyter](http://localhost:8888) though SSH tunnel: **```http://127.0.0.1:8888```**
- <font color="red">If you find **ANY** security vulnerability in this scheme **PLEASE** make me know via GitHub [Issues](https://github.com/chiant1/qmlcourse_walkthrough/issues)</font>